# Custom Training with YOLOv5

In this tutorial, we assemble a dataset and train a custom YOLOv5 model to recognize the objects in our dataset. To do so we will take the following steps:

* Gather a dataset of images and label our dataset
* Export our dataset to YOLOv5
* Train YOLOv5 to recognize the objects in our dataset
* Evaluate our YOLOv5 model's performance
* Run test inference to view our model at work



![](https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png)

# Step 1: Install Requirements

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install wandb
import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 9876, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 9876 (delta 93), reused 138 (delta 91), pack-reused 9734
Receiving objects: 100% (9876/9876), 10.09 MiB | 9.05 MiB/s, done.
Resolving deltas: 100% (6867/6867), done.
/content/yolov5
     |████████████████████████████████| 596 kB 4.4 MB/s 
     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 180 kB 50.6 MB/s 
     |████████████████████████████████| 139 kB 48.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=02479b980116461a6643250acbae2a909421a530b3d9f2c01540d6e016949420
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: 

In [ ]:
import wandb

wandb.login()
from google.colab import drive 
drive.mount('/content/drive',force_remount=False)
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/drive/MyDrive/kickercamds"

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Mounted at /content/drive


In [ ]:
#!curl -L "https://app.roboflow.com/ds/x8MCxicjGs?key=RPrzNjhFFB" > roboflow.zip; unzip roboflow.zip -d /content/drive/MyDrive/kickercamds; rm roboflow.zip

# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!python train.py --noautoanchor --img 428 --rect --batch 128 --epochs 150 --data /content/drive/MyDrive/kickercamds/data.yaml --weights yolov5s.pt --cache

train: Caching images (0.0GB ram):   1% 70/6265 [00:25<37:58,  2.72it/s]
Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/lib/python3.7/linecache.py", line 47, in getlines
    return updatecache(filename, module_globals)
  File "/usr/lib/python3.7/linecache.py", line 137, in updatecache
    lines = fp.readlines()
  File "/usr/lib/python3.7/codecs.py", line 319, in decode
    def decode(self, input, final=False):
KeyboardInterrupt

Original exception was:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 733, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "train.py", line 622, in <module>
    main(opt)
  File "train.py", line 519, in main
    train(opt.hyp, opt, device, callbacks)
  File "train.py", line 213, in train
    prefix=colorstr('train: '))
  File "/content/yolov5/utils/da

KeyboardInterrupt: ignored

#Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 428 --conf 0.1 --source /content/drive/MyDrive/kickercamsmall/test/images

detect: weights=['runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/kickercamsmall/test/images, imgsz=[428, 428], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-25-g15e8c4c torch 1.9.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

Traceback (most recent call last):
  File "detect.py", line 307, in <module>
    main(opt)
  File "detect.py", line 302, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 82, in run
    model = torch.jit.load(w) if 'torchscript' in w else attempt_load(weights, map_location=device)
  File "/content/yolov5/models

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Conclusion and Next Steps

Congratulations! You've trained a custom YOLOv5 model to recognize your custom objects.

To improve you model's performance, we recommend first interating on your datasets coverage and quality. See this guide for [model performance improvement](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results).

To deploy your model to an application, see this guide on [exporting your model to deployment destinations](https://github.com/ultralytics/yolov5/issues/251).

Once your model is in production, you will want to continually iterate and improve on your dataset and model via [active learning](https://blog.roboflow.com/what-is-active-learning/).

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

FileNotFoundError: ignored